In [44]:
import sys

In [1]:
from langchain_community.llms import CTransformers

In [2]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain import ConversationChain
from langchain.chains import ConversationalRetrievalChain

In [3]:
import faiss

In [4]:
loader = CSVLoader(file_path=r'D:\llm-dict\Data\2019.csv')
data = loader.load()

In [5]:
#Split the text inot the chunks 
text_spliter = RecursiveCharacterTextSplitter(chunk_size = 500 , chunk_overlap = 20)

In [6]:
text_chunks = text_spliter.split_documents(data)

In [7]:
embedding = HuggingFaceHubEmbeddings(model = "sentence-transformers/all-MiniLM-L6-v2")

d:\llm-dict\dict_llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
similarity_match = FAISS.from_documents(text_chunks, embedding)

In [9]:
similarity_match.save_local("D:\llm-dict\Data")

In [10]:
docs = similarity_match.similarity_search(query="what is value of GDP percapita of finland provded in the data?")

print(docs)

[Document(page_content='Overall rank: 1\nCountry or region: Finland\nScore: 7.769\nGDP per capita: 1.340\nSocial support: 1.587\nHealthy life expectancy: 0.986\nFreedom to make life choices: 0.596\nGenerosity: 0.153\nPerceptions of corruption: 0.393', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 0}), Document(page_content='Overall rank: 7\nCountry or region: Sweden\nScore: 7.343\nGDP per capita: 1.387\nSocial support: 1.487\nHealthy life expectancy: 1.009\nFreedom to make life choices: 0.574\nGenerosity: 0.267\nPerceptions of corruption: 0.373', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 6}), Document(page_content='Overall rank: 53\nCountry or region: Latvia\nScore: 5.940\nGDP per capita: 1.187\nSocial support: 1.465\nHealthy life expectancy: 0.812\nFreedom to make life choices: 0.264\nGenerosity: 0.075\nPerceptions of corruption: 0.064', metadata={'source': 'D:\\llm-dict\\Data\\2019.csv', 'row': 52}), Document(page_content='Overall rank: 55\nCountry or re

In [11]:
llm = CTransformers(model=r"C:\Users\ganes\Downloads\llama-2-7b-chat.ggmlv3.q4_0.bin", model_type="llama",max_new_token = 512, tempreture = 0.5)

In [12]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever = similarity_match.as_retriever())

In [ ]:

chat_history = []

query = input(f"input prompt .....")

if query == "exit":
    print("Exiting")
    sys.exit()


result = qa({"question": query, "chat_history":chat_history})
print("Response: ", result["answer"])

In [47]:
import pandas as pd

In [49]:
df = pd.read_csv(r"D:\llm-dict\Data\2019.csv")

In [50]:
df

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298
...,...,...,...,...,...,...,...,...,...
151,152,Rwanda,3.334,0.359,0.711,0.614,0.555,0.217,0.411
152,153,Tanzania,3.231,0.476,0.885,0.499,0.417,0.276,0.147
153,154,Afghanistan,3.203,0.350,0.517,0.361,0.000,0.158,0.025
154,155,Central African Republic,3.083,0.026,0.000,0.105,0.225,0.235,0.035


In [8]:
import streamlit as st
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

# Page title
st.set_page_config(page_title='🦜🔗 Ask the Data App')
st.title('🦜🔗 Ask the Data App')

# Load CSV file
def load_csv(input_csv):
  df = pd.read_csv(input_csv)
  with st.expander('See DataFrame'):
    st.write(df)
  return df

# Generate LLM response
def generate_response(csv_file, input_query):
  llm = ChatOpenAI(model_name='gpt-3.5-turbo-0613', temperature=0.2, openai_api_key=openai_api_key)
  df = load_csv(csv_file)
  # Create Pandas DataFrame Agent
  agent = create_pandas_dataframe_agent(llm, df, verbose=True, agent_type=AgentType.OPENAI_FUNCTIONS)
  # Perform Query using the Agent
  response = agent.run(input_query)
  return st.success(response)

# Input widgets
uploaded_file = st.file_uploader('Upload a CSV file', type=['csv'])
question_list = [
  'How many rows are there?',
  'Other']
query_text = st.selectbox('Select an example query:', question_list, disabled=not uploaded_file)
openai_api_key = st.text_input('OpenAI API Key', type='password', disabled=not (uploaded_file and query_text))

# App logic
if query_text is 'Other':
  query_text = st.text_input('Enter your query:', placeholder = 'Enter query here ...', disabled=not uploaded_file)
if not openai_api_key.startswith('sk-'):
  st.warning('Please enter your OpenAI API key!', icon='⚠')
if openai_api_key.startswith('sk-') and (uploaded_file is not None):
  st.header('Output')
  generate_response(uploaded_file, query_text)

<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:37: SyntaxWarning: "is" with a literal. Did you mean "=="?


ImportError: create_pandas_dataframe_agent has been moved to langchain experimental. See https://github.com/langchain-ai/langchain/discussions/11680for more information.
Please update your import statement from: `langchain.agents.create_pandas_dataframe_agent` to `langchain_experimental.agents.create_pandas_dataframe_agent`.

In [7]:
!pip3.9 install altair==4


  Using cached altair-4.0.0-py2.py3-none-any.whl.metadata (12 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
Using cached altair-4.0.0-py2.py3-none-any.whl (709 kB)
Using cached entrypoints-0.4-py3-none-any.whl (5.3 kB)
  Attempting uninstall: altair
    Found existing installation: altair 5.2.0
    Uninstalling altair-5.2.0:
      Successfully uninstalled altair-5.2.0
